In [1]:
!nvidia-smi

Thu Jan 18 01:34:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:C4:00.0 Off |                    0 |
| N/A   33C    P0              62W / 400W |      4MiB / 81920MiB |      0%   E. Process |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']

'0'

In [ ]:
import subprocess



In [117]:
import shlex 
import re

import os
from note_pruning_analysis import open_instruct_dir


def multiline_to_singleline(cmd):
    cmd = cmd.strip()
    cmd = re.sub(r'\\(?![$])', '', cmd) # replace all '\' but not '\$'
    cmd = cmd.replace('\n', '')
    cmd = re.sub(' +', ' ', cmd)
    cmd = cmd.strip()
    return cmd


test_run = 1
test_run = bool(test_run)



torch_dtype = 'bfloat16'
devices = os.environ['CUDA_VISIBLE_DEVICES']
model_name_or_path = '/dccstor/data-pruning/results/oi5_stanford_alpaca:llama-7b/llama-7b_stanford_alpaca_score=random:s=0_pace=prune:size=10000:ep=10'

task_name = 'mtbench_ann=gpt:3.5:turbo:1106'



match = re.search(r'ann=([^_]+)', task_name)
judge_model = match.group(1).replace(':', '-')
from fastchat.model.model_adapter import OPENAI_MODEL_LIST
if not judge_model in OPENAI_MODEL_LIST: # "gpt-4", "gpt-3.5-turbo-1106", "gpt-4-turbo"
    raise ValueError('fastchat does not support the judge model.')
save_dir = f'{model_name_or_path}/eval/{task_name}'
os.makedirs(save_dir, exist_ok=True)

model_id = 'tulu' # for using tulu's template
fastchat_mtbench_data_dir = os.path.normpath(
    os.path.join(open_instruct_dir, '../FastChat/fastchat/llm_judge/data'))
question_file = os.path.join(fastchat_mtbench_data_dir, 'mt_bench/question.jsonl')
answer_file = os.path.join(save_dir, 'model_answer.jsonl')
rating_file = os.path.join(save_dir, f'{judge_model}_single.jsonl')

question_begin, question_end = (0, 1) if test_run else (None, None)
question_begin = 0; question_end = 1

cmd = f"""
    python -m fastchat.llm_judge.gen_model_answer \
        --model-path {model_name_or_path} \
        --model-id {model_id} \
        --bench-name mt_bench \
        --question-file {question_file} \
        {'--question-begin '+str(question_begin) if question_begin else ''} \
        {'--question-end '+str(question_end) if question_end else ''} \
        --max-new-token 2048 \
        --answer-file {answer_file} \
        --dtype {torch_dtype} \
    && \
    python -m fastchat.llm_judge.gen_judgment \
        --bench-name mt_bench \
        --judge-file {os.path.join(fastchat_mtbench_data_dir, 'judge_prompts.jsonl')} \
        --judge-model {judge_model} \
        --mode single \
        --question-file {question_file} \
        --answer-dir {save_dir} \
        --ref-answer-dir {os.path.join(fastchat_mtbench_data_dir, 'mt_bench/reference_answer')} \
        --output-file {rating_file} \
        --first-n 1 \
    && \
    python -m fastchat.llm_judge.show_result \
        --bench-name mt_bench \
        --input-file {rating_file} \
        --mode single \
        --save-to-json
"""
# cmd = 'echo hi && echo foo'
cmd = multiline_to_singleline(cmd)
# cmd = shlex.split(cmd)
cmds = [cmd]




In [114]:
with open('eval_mtbench.sh', 'w') as f:
    s = 'set -e\nset -x\n'
    devices = os.environ['CUDA_VISIBLE_DEVICES'].split(',')[-1]
    s += '\n\n'.join([f"CUDA_VISIBLE_DEVICES={devices} "+x for x in cmds])
    f.write(s)



In [115]:
!cat eval_mtbench.sh

set -e
set -x
CUDA_VISIBLE_DEVICES=0 python -m fastchat.llm_judge.gen_model_answer --model-path /dccstor/data-pruning/results/oi5_stanford_alpaca:llama-7b/llama-7b_stanford_alpaca_score=random:s=0_pace=prune:size=10000:ep=10 --model-id tulu --bench-name mt_bench --question-file /dccstor/data-pruning/wpq/github/mitibm2023/external/FastChat/fastchat/llm_judge/data/mt_bench/question.jsonl --question-end 1 --max-new-token 2048 --answer-file /dccstor/data-pruning/results/oi5_stanford_alpaca:llama-7b/llama-7b_stanford_alpaca_score=random:s=0_pace=prune:size=10000:ep=10/eval/mtbench_ann=gpt:3.5:turbo:1106/model_answer.jsonl --dtype bfloat16 && python -m fastchat.llm_judge.gen_judgment --bench-name mt_bench --judge-file /dccstor/data-pruning/wpq/github/mitibm2023/external/FastChat/fastchat/llm_judge/data/judge_prompts.jsonl --judge-model gpt-3.5-turbo-1106 --mode single --question-file /dccstor/data-pruning/wpq/github/mitibm2023/external/FastChat/fastchat/llm_judge/data/mt_bench/question.jsonl

In [111]:
!bash eval_mtbench.sh

+ CUDA_VISIBLE_DEVICES=0
+ python -m fastchat.llm_judge.show_result --bench-name mt_bench --input-file /dccstor/data-pruning/results/oi5_stanford_alpaca:llama-7b/llama-7b_stanford_alpaca_score=random:s=0_pace=prune:size=10000:ep=10/eval/mtbench_ann=gpt:3.5:turbo:1106/gpt-3.5-turbo-1106_single.jsonl --mode single --save-to-json
Mode: single
Input file: /dccstor/data-pruning/results/oi5_stanford_alpaca:llama-7b/llama-7b_stanford_alpaca_score=random:s=0_pace=prune:size=10000:ep=10/eval/mtbench_ann=gpt:3.5:turbo:1106/gpt-3.5-turbo-1106_single.jsonl

########## First turn ##########
                   score
model        turn       
model_answer 1       7.0

########## Second turn ##########
                   score
model        turn       
model_answer 2       8.0

########## Average ##########
              score
model              
model_answer    7.5
Input file: /dccstor/data-pruning/results/oi5_stanford_alpaca:llama-7b/llama-7b_stanford_alpaca_score=random:s=0_pace=prune:size=10000:ep=1

In [105]:
import pandas as pd
input_file = rating_file


print(f"Input file: {input_file}")
df_all = pd.read_json(input_file, lines=True)
df = df_all[["model", "score", "turn"]]
df = df[df["score"] != -1]
df_all



Input file: /dccstor/data-pruning/results/oi5_stanford_alpaca:llama-7b/llama-7b_stanford_alpaca_score=random:s=0_pace=prune:size=10000:ep=10/eval/mtbench_ann=gpt:3.5:turbo:1106/gpt-3.5-turbo-1106_single.jsonl


,question_id,model,judge,user_prompt,judgment,score,turn,tstamp
0,81,model_answer,"[gpt-3.5-turbo-1106, single-v1]",[Instruction]\nPlease act as an impartial judg...,The response provides an engaging travel blog ...,7,1,1.705565e+09
1,81,model_answer,"[gpt-3.5-turbo-1106, single-v1-multi-turn]",<|The Start of Assistant A's Conversation with...,The response provided by Assistant A is well-s...,8,2,1.705565e+09


In [100]:
df[["model", "score"]].groupby(["model"]).mean().iloc

,score
model,
model_answer,7.5


In [102]:
os.path.dirname(rating_file)

'/dccstor/data-pruning/results/oi5_stanford_alpaca:llama-7b/llama-7b_stanford_alpaca_score=random:s=0_pace=prune:size=10000:ep=10/eval/mtbench_ann=gpt:3.5:turbo:1106'

In [30]:



!cd ../../FastChat/fastchat/llm_judge && CUDA_VISIBLE_DEVICES=0 python gen_model_answer.py \
    --model-path /dccstor/data-pruning/results/oi5_stanford_alpaca:llama-7b/llama-7b_stanford_alpaca_score=random:s=0_pace=prune:size=30000:ep=3 \
    --model-id tulu \
    --question-begin 0 \
    --question-end 10 \
    --max-new-token 2048 \




[2024-01-18 02:06:58,001] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Output to data/mt_bench/model_answer/tulu.jsonl
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
  0%|                                                    | 0/10 [00:00<?, ?it/s]/dccstor/data-pruning/miniconda3/envs/open-instruct/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed

In [5]:
!cd ../../FastChat/fastchat/llm_judge && \
    python gen_judgment.py --model-list tulu --parallel 1 --first-n 1

[2024-01-18 01:36:43,721] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Stats:
{
    "bench_name": "mt_bench",
    "mode": "single",
    "judge": "gpt-4",
    "baseline": null,
    "model_list": [
        "tulu"
    ],
    "total_num_questions": 1,
    "total_num_matches": 2,
    "output_path": "data/mt_bench/model_judgment/gpt-4_single.jsonl"
}
100%|█████████████████████████████████████████████| 2/2 [00:24<00:00, 12.12s/it]


In [27]:
from fastchat.llm_judge.common import Judge
from fastchat.model.model_adapter import get_conversation_template
model = 'gpt-3.5-turbo-1106'
judge = Judge(
        model, judge_prompts["single-v1-multi-turn"], multi_turn=True
    )

conv = get_conversation_template(model)
conv.append_message('user', 'hi')
conv.to_openai_api_messages()

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'hi'}]

In [7]:
print("[Instruction]\nPlease act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: \"[[rating]]\", for example: \"Rating: [[5]]\".\n\n[Question]\nCompose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.\n\n[The Start of Assistant's Answer]\nHawaii is a magical place full of natural beauty and unique cultural experiences. From lush rainforests and verdant peaks to pristine beaches and thriving coral reefs, the islands of Hawaii are a wonder to behold. Travelers can explore the island's diverse ecosystems, learn about the local history and culture, and take in the breathtaking views that make Hawaii such a special place. Here are some of the must-see attractions on the islands of Hawaii, and some of the unique cultural experiences visitors should be sure to check out: \n\n1. Visit the iconic Mauna Kea Volcano, a dormant volcano that is one of the tallest mountains in the world. Take a hike up the mountain to explore the lush landscape, or simply admire the views of the Pacific Ocean from the summit.\n\n2. Take a trip to the famous Waikiki Beach and explore the beautiful beaches and bustling shops.\n\n3. Visit the historic Kauai Lighthouse and take in the stunning views of the Pacific Ocean.\n\n4. Visit the famous I\n[The End of Assistant's Answer]")

[Instruction]
Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".

[Question]
Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.

[The Start of Assistant's Answer]
Hawaii is a magical place full of natural beauty and unique cultural experiences. From lush rainforests and verdant peaks to pristine beaches and thriving coral reefs, the islands of Hawaii are a wonder to behold. Travelers can explore the island's diverse ecosystems, learn about t

In [8]:
print("<|The Start of Assistant A's Conversation with User|>\n\n### User:\nCompose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.\n\n### Assistant A:\nHawaii is a magical place full of natural beauty and unique cultural experiences. From lush rainforests and verdant peaks to pristine beaches and thriving coral reefs, the islands of Hawaii are a wonder to behold. Travelers can explore the island's diverse ecosystems, learn about the local history and culture, and take in the breathtaking views that make Hawaii such a special place. Here are some of the must-see attractions on the islands of Hawaii, and some of the unique cultural experiences visitors should be sure to check out: \n\n1. Visit the iconic Mauna Kea Volcano, a dormant volcano that is one of the tallest mountains in the world. Take a hike up the mountain to explore the lush landscape, or simply admire the views of the Pacific Ocean from the summit.\n\n2. Take a trip to the famous Waikiki Beach and explore the beautiful beaches and bustling shops.\n\n3. Visit the historic Kauai Lighthouse and take in the stunning views of the Pacific Ocean.\n\n4. Visit the famous I\n\n### User:\nRewrite your previous response. Start every sentence with the letter A.\n\n### Assistant A:\nA Hawaii is a magical place full of natural beauty and unique cultural experiences. From lush rainforests and verdant peaks to pristine beaches and thriving coral reefs, the islands of Hawaii are a wonder to behold. Travelers can explore the island's diverse ecosystems, learn about the local history and culture, and take in the breathtaking views that make Hawaii such a special place. Here are some of the must-see attractions on the islands of Hawaii, and some of the unique cultural experiences visitors should be sure to check out: \n\n1. Visit the iconic Mauna Kea Volcano, a dormant volcano that is one of the tallest mountains in the world. Take a hike up the mountain to explore the lush landscape, or simply admire the views of the Pacific Ocean from the summit.\n\n2. Take a trip to the famous Waikiki Beach and explore the beautiful beaches and bustling shops.\n\n3. Visit the historic Kauai Lighthouse and take in the stunning views of the Pacific Ocean.\n\n4. Visit the famous Iol\n\n<|The End of Assistant A's Conversation with User|>")

<|The Start of Assistant A's Conversation with User|>

### User:
Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.

### Assistant A:
Hawaii is a magical place full of natural beauty and unique cultural experiences. From lush rainforests and verdant peaks to pristine beaches and thriving coral reefs, the islands of Hawaii are a wonder to behold. Travelers can explore the island's diverse ecosystems, learn about the local history and culture, and take in the breathtaking views that make Hawaii such a special place. Here are some of the must-see attractions on the islands of Hawaii, and some of the unique cultural experiences visitors should be sure to check out: 

1. Visit the iconic Mauna Kea Volcano, a dormant volcano that is one of the tallest mountains in the world. Take a hike up the mountain to explore the lush landscape, or simply admire the views of the Pacific Ocean from the summit.

2. Take a trip to 

In [12]:
from fastchat.llm_judge.common import load_judge_prompts

judge_file = '/dccstor/data-pruning/wpq/github/mitibm2023/external/FastChat/fastchat/llm_judge/data/judge_prompts.jsonl'
judge_prompts = load_judge_prompts(judge_file)
judge_prompts

/dccstor/data-pruning/miniconda3/envs/open-instruct/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-18 01:42:45,418] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


{'pair-v2': {'name': 'pair-v2',
  'type': 'pairwise',
  'system_prompt': 'Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user\'s instructions and answers the user\'s question better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. Begin your evaluation by comparing the two responses and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]" if assistant B is better, and "[[C

In [18]:
print(judge_prompts['single-v1-multi-turn']['prompt_template'])

<|The Start of Assistant A's Conversation with User|>

### User:
{question_1}

### Assistant A:
{answer_1}

### User:
{question_2}

### Assistant A:
{answer_2}

<|The End of Assistant A's Conversation with User|>


In [1]:
from rosemary import jpt_setup; jpt_setup()

/dccstor/data-pruning/wpq/github/mitibm2023/external/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


In [6]:
import os
from note_pruning_analysis import open_instruct_dir


question_file = os.path.normpath(os.path.join(open_instruct_dir, '../FastChat/fastchat/llm_judge/data/mt_bench/question.jsonl'))
question_file


'/dccstor/data-pruning/wpq/github/mitibm2023/external/FastChat/fastchat/llm_judge/data/mt_bench/question.jsonl'

In [7]:

import torch
from fastchat.llm_judge.common import load_questions, temperature_config
from fastchat.model import load_model, get_conversation_template
from fastchat.llm_judge.gen_model_answer import get_model_answers, run_eval



# model_id = 'llama-7b'
# model_path = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/huggyllama/llama-7b'
model_id = 'tulu'
model_id = 'zephyr-7b'
model_path = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/results/baselines/HuggingFaceH4/zephyr-7b-beta'


model_id = 'tulu'
model_path = '/dccstor/data-pruning/results/oi5_stanford_alpaca:llama-7b/llama-7b_stanford_alpaca_score=random:s=0_pace=prune:size=30000:ep=3'


torch_dtype = 'float16'
torch_dtype = 'bfloat16'
# question_file = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/FastChat/fastchat/llm_judge/data/mt_bench/question.jsonl'


answer_file = f'{model_id}.jsonl'


kwargs = {
    'model_path': model_path,
    'model_id': model_id,
    'question_file': question_file,
    'question_begin': 0,
    'question_end': 1,
    'answer_file': answer_file,
    'max_new_token': 256,
    'num_choices': 1,
    'num_gpus_per_model': 1,
    'num_gpus_total': 1,
    'max_gpu_memory': None,
    'dtype': getattr(torch, torch_dtype),
    'revision': 'main',
}


run_eval(**kwargs)

for k, v in kwargs.items():
    globals()[k] = v


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.26s/it]


KeyboardInterrupt: 

In [ ]:


# questions = load_questions(question_file, question_begin, question_end)
questions = load_questions(question_file, None, None)
# # random shuffle the questions to balance the loading
# random.shuffle(questions)
questions


In [15]:
from collections import Counter

Counter([x['category'] for x in questions])

Counter({'writing': 10,
         'roleplay': 10,
         'reasoning': 10,
         'math': 10,
         'coding': 10,
         'extraction': 10,
         'stem': 10,
         'humanities': 10})

In [11]:

# Split the question file into `num_gpus` files
assert num_gpus_total % num_gpus_per_model == 0
use_ray = num_gpus_total // num_gpus_per_model > 1

if use_ray:
    get_answers_func = ray.remote(num_gpus=num_gpus_per_model)(
        get_model_answers
    ).remote
else:
    get_answers_func = get_model_answers

In [12]:
chunk_size = len(questions) // (num_gpus_total // num_gpus_per_model)
chunk_size

80

In [13]:
model, tokenizer = load_model(
    model_path,
    revision=revision,
    device="cuda",
    num_gpus=num_gpus_per_model,
    max_gpu_memory=max_gpu_memory,
    dtype=dtype,
    load_8bit=False,
    cpu_offloading=False,
    debug=False,
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB (GPU 0; 31.75 GiB total capacity; 1002.02 MiB already allocated; 441.88 MiB free; 1020.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:

from tqdm import tqdm

for question in tqdm(questions):
    if question["category"] in temperature_config:
        temperature = temperature_config[question["category"]]
    else:
        temperature = 0.7

    choices = []
    for i in range(num_choices):
        torch.manual_seed(i)
        conv = get_conversation_template(model_id)
        turns = []
        for j in range(len(question["turns"])):
            qs = question["turns"][j]
            conv.append_message(conv.roles[0], qs)
            conv.append_message(conv.roles[1], None)
            prompt = conv.get_prompt()
            input_ids = tokenizer([prompt]).input_ids

            if temperature < 1e-4:
                do_sample = False
            else:
                do_sample = True

            # some models may error out when generating long outputs
            try:
                output_ids = model.generate(
                    torch.as_tensor(input_ids).cuda(),
                    do_sample=do_sample,
                    temperature=temperature,
                    max_new_tokens=max_new_token,
                )
                if model.config.is_encoder_decoder:
                    output_ids = output_ids[0]
                else:
                    output_ids = output_ids[0][len(input_ids[0]) :]

                # be consistent with the template's stop_token_ids
                if conv.stop_token_ids:
                    stop_token_ids_index = [
                        i
                        for i, id in enumerate(output_ids)
                        if id in conv.stop_token_ids
                    ]
                    if len(stop_token_ids_index) > 0:
                        output_ids = output_ids[: stop_token_ids_index[0]]

                output = tokenizer.decode(
                    output_ids,
                    spaces_between_special_tokens=False,
                )
                if conv.stop_str and isinstance(conv.stop_str, list):
                    stop_str_indices = sorted(
                        [
                            output.find(stop_str)
                            for stop_str in conv.stop_str
                            if output.find(stop_str) > 0
                        ]
                    )
                    if len(stop_str_indices) > 0:
                        output = output[: stop_str_indices[0]]
                elif conv.stop_str and output.find(conv.stop_str) > 0:
                    output = output[: output.find(conv.stop_str)]

                for special_token in tokenizer.special_tokens_map.values():
                    if isinstance(special_token, list):
                        for special_tok in special_token:
                            output = output.replace(special_tok, "")
                    else:
                        output = output.replace(special_token, "")
                output = output.strip()

                if conv.name == "xgen" and output.startswith("Assistant:"):
                    output = output.replace("Assistant:", "", 1).strip()
            except RuntimeError as e:
                print("ERROR question ID: ", question["question_id"])
                output = "ERROR"

            conv.update_last_message(output)
            turns.append(output)

        choices.append({"index": i, "turns": turns})

#     # Dump answers
#     os.makedirs(os.path.dirname(answer_file), exist_ok=True)
#     with open(os.path.expanduser(answer_file), "a") as fout:
#         ans_json = {
#             "question_id": question["question_id"],
#             "answer_id": shortuuid.uuid(),
#             "model_id": model_id,
#             "choices": choices,
#             "tstamp": time.time(),
#         }
#         fout.write(json.dumps(ans_json) + "\n")
